In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
from dataclasses import dataclass

@dataclass
class TrainCfg:
    batch_size: int = 128
    max_epochs: int = 150
    lr: float = 3e-4
    device: str = "cuda" if torch.cuda.is_available() else "cpu"


CFG = TrainCfg()

In [4]:
class CardioRiskNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return torch.sigmoid(self.fc3(x))


In [ ]:
import torch
from torch.utils.data import Dataset
import random

class SyntheticPatientDataset(Dataset):
    def __init__(self, num_samples=5000, device="cpu"):
        self.X = []
        self.y = []

        for _ in range(num_samples):
            r = random.random()

            # -------- Regime Sampling --------
            if r < 0.25:  # LOW RISK
                ecg_prob = random.uniform(0.0, 0.1)
                exercise = random.uniform(0.85, 1.0)
                diet = random.uniform(0.85, 1.0)
                sleep = random.uniform(0.85, 1.0)
                smoking = random.randint(0, 5)
                alcohol = random.randint(0, 3)
                age = random.uniform(20, 35)
                sys_bp = random.uniform(100, 120)
                dia_bp = random.uniform(65, 80)
                height_cm = random.uniform(160, 185)
                weight_kg = random.uniform(55, 80)

            elif r < 0.75:  # MID RISK
                ecg_prob = random.uniform(0.2, 0.6)
                exercise = random.uniform(0.4, 0.7)
                diet = random.uniform(0.4, 0.7)
                sleep = random.uniform(0.4, 0.7)
                smoking = random.randint(5, 60)
                alcohol = random.randint(3, 15)
                age = random.uniform(35, 60)
                sys_bp = random.uniform(120, 150)
                dia_bp = random.uniform(80, 95)
                height_cm = random.uniform(155, 185)
                weight_kg = random.uniform(70, 100)

            else:  # HIGH RISK
                ecg_prob = random.uniform(0.7, 1.0)
                exercise = random.uniform(0.0, 0.3)
                diet = random.uniform(0.0, 0.3)
                sleep = random.uniform(0.0, 0.3)
                smoking = random.randint(60, 140)
                alcohol = random.randint(15, 30)
                age = random.uniform(60, 80)
                sys_bp = random.uniform(150, 180)
                dia_bp = random.uniform(95, 120)
                height_cm = random.uniform(155, 185)
                weight_kg = random.uniform(90, 130)

            sex = random.choice([0, 1])  # 0 = male, 1 = female

            age_n = (age - 20) / 60
            smoke_n = min(smoking / 140, 1.0)
            alcohol_n = min(alcohol / 30, 1.0)
            sys_n = (sys_bp - 90) / 90
            dia_n = (dia_bp - 60) / 60

            bmi = weight_kg / (height_cm / 100) ** 2
            bmi_n = max(0.0, min((bmi - 18.5) / 16.5, 1.0))

            base_risk = (
                0.40 * ecg_prob +
                0.10 * age_n +
                0.10 * bmi_n +
                0.10 * sys_n +
                0.05 * dia_n +
                0.05 * smoke_n +
                0.05 * alcohol_n +
                0.075 * (1 - exercise) +
                0.075 * (1 - diet) +
                0.075 * (1 - sleep)
            )

            interactions = (
                0.25 * max(0.0, age_n - 0.35) * smoke_n +
                0.20 * max(0.0, sys_n - 0.45) * max(0.0, dia_n - 0.45) +
                0.20 * ecg_prob * max(0.0, sys_n - 0.4) +
                0.05 * sex * max(0.0, age_n - 0.5)  # female age interaction (subtle)
            )

            raw_risk = base_risk + interactions
            raw_risk = max(0.0, min(raw_risk, 1.0))
            risk = raw_risk ** 1.7

            features = torch.tensor([
                ecg_prob,
                exercise, diet, sleep,
                smoke_n, alcohol_n,
                age_n,
                sex,
                bmi_n,
                sys_n, dia_n
            ], dtype=torch.float32)

            self.X.append(features)
            self.y.append(torch.tensor(risk, dtype=torch.float32))

        self.X = torch.stack(self.X).to(device)
        self.y = torch.stack(self.y).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [125]:
def train_model(model, dataset, cfg=CFG):
    loader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
    criterion = nn.MSELoss()

    for epoch in range(cfg.max_epochs):
        total_loss = 0
        model.train()
        for X, y in loader:
            optimizer.zero_grad()
            pred = model(X).squeeze()
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{cfg.max_epochs}, Loss: {total_loss/len(loader):.4f}")
    return model



In [127]:
ds = SyntheticPatientDataset(10000)

print("Min risk:", ds.y.min().item())
print("Mean risk:", ds.y.mean().item())
print("Max risk:", ds.y.max().item())

model = CardioRiskNet(input_size=11).to(CFG.device)

model = train_model(model, ds)



Min risk: 0.004319293890148401
Mean risk: 0.41104400157928467
Max risk: 1.0
Epoch 1/150, Loss: 0.1254
Epoch 2/150, Loss: 0.0694
Epoch 3/150, Loss: 0.0215
Epoch 4/150, Loss: 0.0101
Epoch 5/150, Loss: 0.0079
Epoch 6/150, Loss: 0.0067
Epoch 7/150, Loss: 0.0057
Epoch 8/150, Loss: 0.0052
Epoch 9/150, Loss: 0.0046
Epoch 10/150, Loss: 0.0043
Epoch 11/150, Loss: 0.0036
Epoch 12/150, Loss: 0.0032
Epoch 13/150, Loss: 0.0029
Epoch 14/150, Loss: 0.0026
Epoch 15/150, Loss: 0.0024
Epoch 16/150, Loss: 0.0022
Epoch 17/150, Loss: 0.0020
Epoch 18/150, Loss: 0.0019
Epoch 19/150, Loss: 0.0017
Epoch 20/150, Loss: 0.0016
Epoch 21/150, Loss: 0.0015
Epoch 22/150, Loss: 0.0014
Epoch 23/150, Loss: 0.0013
Epoch 24/150, Loss: 0.0013
Epoch 25/150, Loss: 0.0012
Epoch 26/150, Loss: 0.0012
Epoch 27/150, Loss: 0.0012
Epoch 28/150, Loss: 0.0011
Epoch 29/150, Loss: 0.0011
Epoch 30/150, Loss: 0.0010
Epoch 31/150, Loss: 0.0010
Epoch 32/150, Loss: 0.0010
Epoch 33/150, Loss: 0.0009
Epoch 34/150, Loss: 0.0009
Epoch 35/150, L

In [130]:
model.eval()

torch.save(model.state_dict(), "cardiorisknet_model.pt")

In [128]:
age, smoking_per_week, alcohol_per_week = 20, 0, 0
height_cm, weight_kg = 175, 65
sys_bp, dia_bp = 110, 70

age_norm = (age - 20) / 60
smoking_norm = smoking_per_week / 140
alcohol_norm = alcohol_per_week / 30

height_m = height_cm / 100.0
bmi = weight_kg / (height_m ** 2)
bmi_norm = (bmi - 18.5) / 16.5
bmi_norm = max(0.0, min(bmi_norm, 1.0))

sys_bp_norm = (sys_bp - 90) / 90
dia_bp_norm = (dia_bp - 60) / 60


In [129]:
model(torch.tensor([0.13, 1, 1, 1, smoking_norm, alcohol_norm, age_norm, 1, bmi_norm, sys_bp_norm, dia_bp_norm]))

tensor([0.0224], grad_fn=<SigmoidBackward0>)